In [ ]:
#append to same df
#add field for category (with unique url)

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def get_section_title(soup_obj):
        title_str = soup_obj.find('span',class_='title').get_text()
        return title_str

def get_data_wrapper(section):
    data_wrappers = section.select('div.data-wrapper')
    return data_wrappers

def get_data_header_text(wrapper):
    data_header_text = wrapper.select('div.data-header')[0].get_text()
    return data_header_text

def get_data_value_text(wrapper):
    data_value_text = wrapper.select('div.data-cell')[0].get_text(strip=True).replace("\r\n","")
    return data_value_text

def get_data_row(soup_obj):
    data_row = soup_obj.find_all('div',class_='data-cell')
    return data_row


urls = {'Gun Deer - Antlerless/Spike':'https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GDA',
        'Gun Deer - Either Sex':'https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GDE',
        'Feral Hog':'https://tpwd.texas.gov/huntwild/hunt/public/public_hunt_drawing/hunt-category-details.phtml?OCat=GFH'}


items = urls.items()



In [83]:
df4 = pd.DataFrame()

for category, url in items:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    sections = soup.find_all('section')

    #Generate dataframe for fields with one data-cell per data-row
    list_of_dicts = []
    for section in sections:
        data_wrappers = get_data_wrapper(section)
        d = {}
        d['Title'] = get_section_title(section)
        for wrapper in data_wrappers:
            for cell in data_wrappers:
                d[get_data_header_text(wrapper)] = get_data_value_text(wrapper)

        list_of_dicts.append(d)

    
    df1 = pd.DataFrame.from_records(list_of_dicts)
    
    
    # generate dataframe for Permits, Age Requirements, and Last Year, which have multiple data-cells per row
    def split_data_cell(data_cell):
        pair = data_cell.split(':')
        return pair

    keywords = ['Available', 'Fee', 'Youth ages', 'Adult minimum', 'Supervising adult', 'Applicants', 'Permits/Groups', 'Success Rate']


    list_of_dicts1 = []
    for section in sections:
        data_wrappers = get_data_wrapper(section)
        d = {}
        for wrapper in data_wrappers:
                data_row = get_data_row(wrapper)
                for cell in data_row:
                    for i in keywords:
                        if i in cell.get_text():
                            split_values = cell.get_text(strip=True).split(':')
                            key = split_values[0].replace("\r\n","")
                            value = split_values[1].replace("\r\n","")
                            d[key] = value
                        else:
                            continue
        list_of_dicts1.append(d)

    df2 = pd.DataFrame.from_records(list_of_dicts1)

        #join the two dataframes
    df3 = pd.concat([df1.reset_index(drop=True),df2.reset_index(drop=True)], axis=1)
    df3


    #clean fields
    df3['Hunt Dates'] = df3['Hunt Dates'].str.replace(' ','')
    df3['Hunt Dates'] = df3['Hunt Dates'].str.replace('–','; ')
    df3['Youth ages'] = df3['Youth ages'].str.replace(' ','')
    df3['Youth ages'] = df3['Youth ages'].str.replace('-',' to ')

    #drop fields
    try:
        df3 = df3.drop(columns=['Groups'])
    except:
        pass
    try:
        df3 = df3.drop(columns=['Last Year'])
    except:
        pass
    try:
        df3 = df3.drop(columns=['Age Requirements'])
    except:
        pass
    try:
        df3 = df3.drop(columns=['Permits'])
    except:
        pass

    #add fields
    df3['Available'] = df3['Available'].astype(float)
    df3['Applicants'] = df3['Applicants'].astype(float)
    df3["Draw Probability"] = (df3["Available"] / df3["Applicants"])
    
    #append to dataframe
    df4 = df4.append(df3)


In [84]:
#generate csv
df4.to_csv('hunt_data.csv')